# 推文笔记
## 1&2 两阶段鲁棒优化问题

常见鲁棒优化包括：基本鲁棒优化、多阶段鲁棒优化、分布式鲁棒优化。

鲁棒优化的目的是处理优化问题中参数不确定的挑战，致力于优化最坏情况的解，或者最坏参数分布下的解。

两阶段鲁棒优化的问题描述：
- 第一阶段决策参数是确定的，第一阶段决策需要首先做出
- 第二阶段的决策有一些参数是不确定的，需要第一阶段决策确定后，且第二阶段参数揭示后，再做出第二阶段决策
- 两阶段鲁棒优化的目标是，对两阶段进行联合优化，同时考虑第二阶段参数的不确定性，优化第二阶段参数取到最坏情况下的总体目标值。


两阶段鲁棒优化（two-stage RO）问题一般形式为：

\begin{align*}
\underset{y}{\min}\quad &\mathbf{c}^T\mathbf{y}+\underset{u\in \mathcal{U}}{\max}\underset{\mathbf{x}\in \mathbf{F}(\mathbf{y},u)}{\min}\mathbf{b}^T\mathbf{x}
\\
s.t. \quad &\mathbf{Ay}\geqslant \mathbf{d}
\\
&\mathbf{Gx}\geqslant \mathbf{h}-\mathbf{Ey}-\mathbf{M}u\qquad   \mathrm{u}\in \mathcal{U} 
\\
&\mathbf{S}_{\mathbf{y}}\subseteq \mathbb{R} _{+}^{n}
\\
&\mathbf{S}_{\mathbf{x}}\subseteq \mathbb{R} _{+}^{n}
\end{align*}

## 3 Benders dual cutting plane method

我们可以使用Benders dual cutting plane算法求解上述问题，与Benders分解类似，该算法将上述两阶段鲁棒分解为：

- 一个主问题，包含：1第一阶段决策变量$\bf{y}$和只和$\bf{y}$相关的约束，2子问题返回的割平面，3评估第二阶段目标函数的辅助变量$\eta$:


\begin{align*}
\underset{y}{\min}\quad &\mathbf{c}^T\mathbf{y}+\eta
\\
s.t. \quad &\mathbf{Ay}\geqslant \mathbf{d}
\\
&\eta \geqslant0
\\
&\mathbf{S}_{\mathbf{y}}\subseteq \mathbb{R} _{+}^{n}
\end{align*}

- 一个子问题，包含：1第二阶段决策变量$\bf{x}$和不确定变量$\bf{u}$，旨在给出第二阶段目标函数的一个界限:

\begin{align*}
\underset{u\in \mathcal{U}}{\max}\underset{\mathbf{x}\in \mathbf{F}(\mathbf{y},u)}{\min}\quad &\mathbf{b}^T\mathbf{x}
\\
s.t. \quad &\mathbf{G}\bf{x}\geqslant \mathbf{h}-\mathbf{E}{\mathbf{y}}-\mathbf{M}u \qquad  \mathrm{u}\in \mathcal{U} 
\\
&\mathbf{S}_{\mathbf{x}}\subseteq \mathbb{R} _{+}^{n}
\end{align*}

对于max min这种形式的问题我们是无法直接求解的。二阶段子问题中的内层（inner-level）模型是线性规划问题，我们可以写成其对偶形式：

\begin{align*}
\underset{u}{\max}\;\underset{\pi}{\max}\quad &(\mathbf{h}-\mathbf{E}\overline{\mathbf{y}}-\mathbf{M}u)^T\pi
\\
s.t. \quad &\mathbf{G}^T\pi \leqslant \mathbf{b}
\\
&\pi \geqslant \mathbf{0}
\\
&\mathbf{u} \in \mathcal{U}
\end{align*}

在固定了变量$\overline{\mathbf{y}}$的情况下，上述问题求解后为第二阶段的目标函数提供了一个下界。我们可以将改模型简写为：
$$
\bf{\mathcal{Q}} (\mathbf{y})=\underset{\pi ,u}{\max}\left\{ \left( \mathbf{h}-\mathbf{E}\overline{\mathbf{y}}-\mathbf{M}u \right) ^T\pi : \mathbf{G}^T\pi \leqslant \mathbf{b}, u\in \mathcal{U} , \pi \geqslant \mathbf{0} \right\}
$$

子问题$\bf{\mathcal{Q}}(\mathbf{y})$的求解可以大致可以使用以下四种方法：
- 启发式
- 根据$U$的特殊结构设计精确算法
- 使用Gurobi等求解器直接求解
- 使用KKT条件进行求解

求解子问题可以为主问题提供割平面，从而改进全局界限，改进当前解。

割平面的形式如下：

$$\eta\geqslant(\bf{h}-\bf{E}\bf{y}-\bf{M}u^*_k)^T\pi^*_k$$

该割平面可以看做benders分解中的optimality cut，为了提升主问题的下界。

在Benders-dual method迭代过程中，全局上界和下界的计算方法如下：
- $LB=max \; \{ \bf{c}^T\bf{y}^*_{k+1}+\eta^*_{k+1} \}$
- $UB=min \; \{ \bf{c}^T\bf{y}^*_{k}+\bf{Q}(\bf{y}^*_k) \}$

## 4 Column-and-constraint generation algorithm

基本思想如下：
> 首先，只考虑第一阶段的决策变量和只包含第一阶段决策变量的约束，这个版本可看做是整个问题的松弛版本。然后，再想办法将松弛部分的影响逐渐加回来。

> 第二阶段中，求解的目标是找到worst case，再将worst case下的参数和约束代入主问题中即可等价地解决two-stage RO问题。一般无法直接找到worst case，不过我们可以固定一阶段的决策变量$\overline{\bf{y}}$，找到最有可能的worst case，将该场景下的决策变量和约束加入主问题中。通过不断迭代，主问题考虑的可能是最坏情况的场景越来越多，解就会逐步改进直至最优。

根据算法描述，对问题进行如下reformulation：

\begin{align*}
\underset{y}{\min}\quad &\mathbf{c}^T\mathbf{y}+\eta
\\
s.t. \quad &\mathbf{Ay}\geqslant \mathbf{d}
\\
&\eta \geqslant \bf{b}^T\bf{x}^l &&l=1,...,r
\\
&\bf{E}\bf{y}+\bf{G}\bf{x}^l\geqslant \bf{h}-\bf{M}u_l &&l=1,...,r
\\
&\bf{y}\in\mathbf{S}_{\mathbf{y}}
\\
&\bf{x}^l\in\mathbf{S}_{\mathbf{x}} &&l=1,...,r
\end{align*}

其中，**行生成**指不断添加新的约束：
\begin{align*}
&\eta \geqslant \bf{b}^T\bf{x}^l &&l=1,...,r
\\
&\bf{E}\bf{y}+\bf{G}\bf{x}^l\geqslant \bf{h}-\bf{M}u_l &&l=1,...,r
\end{align*}

而**列生成**指不断添加新的一组决策变量，即新的向量$\bf{x}^l$：
\begin{align*}
\bf{x}^l\in\mathbf{S}_{\mathbf{x}} &&l=1,...,r
\end{align*}


In [1]:
# 案例实战 - 部分代码

from gurobipy import *
import numpy as np

""" The input parameter """
facility_num = 3
customer_num = 3
fixed_cost = [400, 414, 326]
unit_capacity_cost = [18, 25, 20]
trans_cost = [[22, 33, 24],
              [33, 23, 30],
              [20, 25, 27]]
max_capacity = 800

demand_nominal = [206, 274, 220]
demand_var = [40, 40, 40]

""" build initial master problem """
""" Create variables """
master = Model('master problem')
x_master = {}
z = {}
y = {}
eta = master.addVar(lb=0, vtype=GRB.CONTINUOUS, name='eta')

for i in range(facility_num):
    y[i] = master.addVar(vtype=GRB.BINARY, name='y_%s' % (i))
    z[i] = master.addVar(lb=0, vtype=GRB.CONTINUOUS, name='z_%s' % (i))

""" Set objective """
obj = LinExpr()
for i in range(facility_num):
    obj.addTerms(fixed_cost[i], y[i])
    obj.addTerms(unit_capacity_cost[i], z[i])
obj.addTerms(1, eta)

master.setObjective(obj, GRB.MINIMIZE)

""" Add Constraints  """
# cons 1
for i in range(facility_num):
    master.addConstr(z[i] <= max_capacity * y[i])

""" Add initial value Constraints  """
# create new variables x
iter_cnt = 0
for i in range(facility_num):
    for j in range(customer_num):
        x_master[iter_cnt, i, j] = master.addVar(lb=0
                                                 , ub=GRB.INFINITY
                                                 , vtype=GRB.CONTINUOUS
                                                 , name='x_%s_%s_%s' % (iter_cnt, i, j))
# create new constraints
expr = LinExpr()
for i in range(facility_num):
    for j in range(customer_num):
        expr.addTerms(trans_cost[i][j], x_master[iter_cnt, i, j])
master.addConstr(eta >= expr)

expr = LinExpr()
for i in range(facility_num):
    expr.addTerms(1, z[i])
master.addConstr(expr >= 772)  # 206 + 274 + 220 + 40 * 1.8

""" solve the model and output """
master.optimize()
print('Obj = {}'.format(master.ObjVal))
print('-----  location ----- ')
for key in z.keys():
    print('facility : {}, location: {}, capacity: {}'.format(key, y[key].x, z[key].x))


def print_sub_sol(model, d, g, x):
    d_sol = {}
    if(model.status != 2):
        print('The problem is infeasible or unbounded!')
        print('Status: {}'.format(model.status))
        d_sol[0] = 0
        d_sol[1] = 0
        d_sol[2] = 0
    else:
        print('Obj(sub) : {}'.format(model.ObjVal), end='\t | ')
        for key in d.keys():
            # print('demand: {}, perturbation = {}'.format(d[key].x, g[key].x))
            d_sol[key] = d[key].x
    return d_sol


""" Column-and-constraint generation """

LB = -np.inf
UB = np.inf
iter_cnt = 0
max_iter = 30
cut_pool = {}
eps = 0.001
Gap = np.inf

z_sol = {}
for key in z.keys():
    z_sol[key] = z[key].x
# print(z_sol)

""" solve the master problem and update bound """
master.optimize()

""" 
 Update the Lower bound 
"""
LB = master.ObjVal
print('LB: {}'.format(LB))

''' create the subproblem '''
subProblem = Model('sub problem')
""" set objective """
sub_obj = LinExpr()
for i in range(facility_num):
    for j in range(customer_num):
        sub_obj.addTerms(trans_cost[i][j], x[i, j])
subProblem.setObjective(sub_obj, GRB.MAXIMIZE)

""" add constraints to subproblem """
# cons 1
for i in range(facility_num):
    expr = LinExpr()
    for j in range(customer_num):
        expr.addTerms(1, x[i, j])
    subProblem.addConstr(expr <= z_sol[i], name='sub_capacity_1_z_%s' % i)

# cons 2
for j in range(facility_num):
    expr = LinExpr()
    for i in range(customer_num):
        expr.addTerms(1, x[i, j])
    subProblem.addConstr(expr >= d[j])

# cons 3
for i in range(facility_num):
    for j in range(customer_num):
        subProblem.addConstr(pi[i] - theta[j] <= trans_cost[i][j])

""" demand constraints """
for j in range(customer_num):
    subProblem.addConstr(d[j] == demand_nominal[j] + g[j] * demand_var[j])

subProblem.addConstr(g[0] + g[1] + g[2] <= 1.8)
subProblem.addConstr(g[0] + g[1] <= 1.2)

""" logic constraints """
# logic 1
....

# logic 2
....

# logic 3
....

subProblem.write('SP.lp')
subProblem.optimize()
d_sol = {}

print('\n\n\n *******            C&CG starts          *******  ')
print('\n **                Initial Solution             ** ')

d_sol = print_sub_sol(subProblem, d, g, x)

""" 
 Update the initial Upper bound 
"""
# UB = min(UB, subProblem.ObjVal + master.ObjVal - eta.x)
# print('UB (iter {}): {}'.format(iter_cnt, UB))

# close the outputflag
master.setParam('Outputflag', 0)
subProblem.setParam('Outputflag', 0)
"""
 Main loop of CCG algorithm 
"""
while (UB - LB > eps and iter_cnt <= max_iter):
    iter_cnt += 1
    # print('\n\n --- iter : {} --- \n'.format(iter_cnt))
    print('\n iter : {} '.format(iter_cnt), end='\t | ')

    # create new variables x
    for i in range(facility_num):
        for j in range(customer_num):
            x_master[iter_cnt, i, j] = master.addVar(lb=0
                                                     , ub=GRB.INFINITY
                                                     , vtype=GRB.CONTINUOUS
                                                     , name='x_%s_%s_%s' % (iter_cnt, i, j))

    # if subproblem is frasible and bound, create variables xk+1 and add the new constraints
    if (subProblem.status == 2 and subProblem.ObjVal < 1000000000):
        # create new constraints
        expr = LinExpr()
        for i in range(facility_num):
            for j in range(customer_num):
                expr.addTerms(trans_cost[i][j], x_master[iter_cnt, i, j])
        master.addConstr(eta >= expr)

        # create worst case related constraints
        # cons 2
        ....

        # cons 3
        ....

        # solve the resulted master problem
        master.optimize()
        print('Obj(master): {}'.format(master.ObjVal), end='\t | ')
        """ Update the LB """
        LB = master.ObjVal
        print('LB (iter {}): {}'.format(iter_cnt, LB), end='\t | ')

        """ Update the subproblem """
        # first, get z_sol from updated master problem
        for key in z.keys():
            z_sol[key] = z[key].x

        # change the coefficient of subproblem
        ....
        
        # add new constraints
        # cons 1
        ....

        # logic 1
        ....

        """ Update the lower bound """
        subProblem.optimize()
        d_sol = print_sub_sol(subProblem, d, g, x)

        """ 
         Update the Upper bound 
        """
        if (subProblem.status == 2):
            UB = min(UB, subProblem.ObjVal + master.ObjVal - eta.x)
            # print('eta = {}'.format(eta.x))
        print('UB (iter {}): {}'.format(iter_cnt, UB), end='\t | ')
        Gap = round(100 * (UB - LB) / UB, 2)
        print('eta = {}'.format(eta.x), end='\t | ')
        print(' Gap: {} %  '.format(Gap), end='\t')

        # If the subproblem is unbounded
    if (subProblem.status == 4):
        # create worst case related constraints
        # cons 2
        ....

        # cons 3
        ....

        # solve the resulted master problem
        master.optimize()
        print('Obj(master): {}'.format(master.ObjVal))

        """ Update the LB """
        LB = master.ObjVal
        print('LB (iter {}): {}'.format(iter_cnt, LB))

        """ Update the subproblem """
        # first, get z_sol from updated master problem
        for key in z.keys():
            z_sol[key] = z[key].x

        # change the coefficient of subproblem
        ....
        
        # add new constraints
        # cons 1
        ....

        # logic 1
        ....

        """ Update the lower bound """
        subProblem.optimize()
        d_sol = print_sub_sol(subProblem, d, g, x)

        """ 
         Update the Upper bound 
        """
        if (subProblem.status == 2):
            UB = min(UB, subProblem.ObjVal + master.ObjVal - eta.x)
            print('eta = {}'.format(eta.x))
        print('UB (iter {}): {}'.format(iter_cnt, UB))
        Gap = round(100 * (UB - LB) / UB, 2)
        print('---- Gap: {} % ---- '.format(Gap))

master.write('finalMP.lp')
print('\n\nOptimal solution found !')
print('Opt_Obj : {}'.format(master.ObjVal))
print(' **  Final Gap: {} %  **  '.format(Gap))
print('\n  **    Solution  **  ')
for i in range(facility_num):
    print(' {}: {},\t  {}: {} '.format(y[i].varName, y[i].x, z[i].varName, z[i].x), end='')
    print('\t actual demand: {}: {}'.format(d[i].varName, d[i].x), end='')
    print('\t perturbation in worst case: {}: {}'.format(g[i].varName, g[i].x))
print('\n  **    Transportation solution  **  ')
for i in range(facility_num):
    for j in range(customer_num):
        if (x[i, j].x > 0):
            print('trans: {}: {}, cost : {} \t '.format(x[i, j].varName, x[i, j].x, trans_cost[i][j] * x[i, j].x),
                  end='')
    print()